In [245]:
# Loasklearn.linear_modelimport pandas as pd
import numpy as np
import pandas as pd

# Preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

# Cross validation
from sklearn.model_selection import cross_validate

# Bayes opt
from bayes_opt import BayesianOptimization

# Linear Regression with Lasso
from sklearn.linear_model import Lasso
# Linear Regression with L2
from sklearn.linear_model import Ridge
# Random Forest
from sklearn.ensemble import RandomForestRegressor
# XGBoost
import xgboost as xgb
# LightGBM
import lightgbm as lgbm

# Scoring
from sklearn.metrics import mean_squared_error

In [2]:
train_data  = pd.read_csv("../data/processed/train.csv")

In [3]:
train_data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,4,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,2,11
2,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,0,14
3,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,0,13
4,GP,M,16,U,LE3,T,4,3,services,other,...,yes,no,5,4,2,1,2,5,6,13


In [4]:
X_train = train_data.drop(["G3"], axis = 1)
y_train = train_data["G3"]

In [5]:
X_train.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences'],
      dtype='object')

In [7]:
# Defining transformation steps
numeric_transformer = Pipeline(steps=[
    # For standard scaling of data
    ('scaler', StandardScaler())
    ])
categorical_transformer = Pipeline(steps=[
    # OHE for categorical data
    ('onehot', OneHotEncoder(drop = "first"))])

In [41]:
# Identify numerical vs categorical features
categorical_features = X_train.loc[:,("school","sex", "address", "famsize", "Pstatus", "Mjob", "Fjob", "reason", 
                                  "guardian","schoolsup", "famsup", "paid","activities","nursery", "higher", 
                                  "internet","romantic")].columns

numeric_features = X_train.loc[:,("age", "Medu", "Fedu", "traveltime", "studytime", "failures", "famrel", 
                                    "freetime", "goout", "Dalc", "Walc", "health", "absences")].columns


In [42]:
categorical_features

Index(['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob',
       'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities',
       'nursery', 'higher', 'internet', 'romantic'],
      dtype='object')

In [43]:
numeric_features

Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences'],
      dtype='object')

In [67]:
# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('ohe', OneHotEncoder(drop = "first"), categorical_features)
    ])

In [68]:
# Convert to dataframe
X_train_trans = pd.DataFrame(preprocessor.fit_transform(X_train),
                            index = X_train.index,
                             columns = (list(numeric_features) +
                                       list(preprocessor.named_transformers_['ohe'].get_feature_names(categorical_features))))

# Linear Model Ridge

In [129]:
lmridge_params = {'alpha':(0,100)}

def cv_mse_lmridge(alpha):
    """ 
    Performs cross validation for LM regressor with Ridge regression. To be used for Bayesian optimiser maximizer function.
    
    Parameters
    ----------
    alpha : float
        L2 regularisation constant

    Returns
    -------
    float
        Cross validation score based on negative mean squared error.
        
    """
    estimator = Ridge(alpha)

    # Note that neg_mean_squared_error is opposite, thus a negative sign is added for Minimisation Optimisation via BayesOpt
    return cross_validate(estimator, X_train_trans, y_train, cv = 10, scoring = "neg_root_mean_squared_error")["test_score"].mean()

In [130]:
optimizer_lmridge = BayesianOptimization(cv_mse_lmridge, lmridge_params)
optimizer_lmridge.maximize(n_iter = 20)

|   iter    |  target   |   alpha   |
-------------------------------------
|  1        | -2.735    |  72.45    |
|  2        | -2.746    |  13.06    |
|  3        | -2.737    |  94.96    |
|  4        | -2.735    |  55.57    |
|  5        | -2.739    |  28.27    |
|  6        | -2.735    |  63.32    |
|  7        | -2.735    |  63.32    |
|  8        | -2.735    |  61.72    |
|  9        | -2.735    |  68.61    |
|  10       | -2.735    |  55.83    |
|  11       | -2.735    |  70.74    |
|  12       | -2.735    |  56.32    |
|  13       | -2.735    |  70.37    |
|  14       | -2.735    |  56.45    |
|  15       | -2.735    |  57.6     |
|  16       | -2.735    |  71.25    |
|  17       | -2.735    |  57.41    |
|  18       | -2.737    |  100.0    |
|  19       | -2.735    |  70.32    |
|  20       | -2.735    |  56.55    |
|  21       | -2.735    |  69.56    |
|  22       | -2.735    |  56.59    |
|  23       | -2.735    |  68.63    |
|  24       | -2.735    |  56.27    |
|  25       

In [131]:
optimizer_lmridge.max

{'target': -2.73504227729081, 'params': {'alpha': 61.716008089193764}}

# Linear Model Lasso

In [132]:
lmlasso_params = {'alpha':(0,100)}

def cv_mse_lmlasso(alpha):
    """ 
    Performs cross validation for LM regressor with Lasso regression. To be used for Bayesian optimiser maximizer function.
    
    Parameters
    ----------
    alpha : float
        L1 regularisation constant

    Returns
    -------
    float
        Cross validation score based on negative mean squared error.
        
    """
    estimator = Lasso(alpha)

    # Note that neg_mean_squared_error is opposite, thus a negative sign is added for Minimisation Optimisation via BayesOpt
    return cross_validate(estimator, X_train_trans, y_train, cv = 10, scoring = "neg_root_mean_squared_error")["test_score"].mean()

In [133]:
optimizer_lmlasso = BayesianOptimization(cv_mse_lmlasso, lmlasso_params, random_state = 1)
optimizer_lmlasso.maximize(n_iter = 20)

|   iter    |  target   |   alpha   |
-------------------------------------
|  1        | -3.2      |  70.12    |
|  2        | -3.2      |  85.44    |
|  3        | -3.2      |  31.35    |
|  4        | -3.2      |  46.36    |
|  5        | -3.2      |  62.58    |
|  6        | -2.757    |  0.003969 |


/home/kenneth/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/home/kenneth/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/home/kenneth/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1734.3718561958653, tolerance: 0.5434673076923078
  positive)
/home/kenneth/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train,

|  7        | -2.77     |  0.0      |
|  8        | -2.98     |  0.638    |
|  9        | -3.2      |  19.45    |
|  10       | -3.2      |  93.82    |
|  11       | -3.2      |  54.56    |
|  12       | -3.2      |  38.55    |
|  13       | -3.2      |  9.394    |
|  14       | -3.2      |  78.23    |
|  15       | -3.2      |  100.0    |
|  16       | -3.2      |  25.4     |
|  17       | -3.2      |  14.42    |
|  18       | -3.2      |  5.097    |
|  19       | -3.2      |  89.63    |
|  20       | -3.2      |  50.46    |
|  21       | -3.2      |  74.17    |
|  22       | -3.2      |  58.57    |
|  23       | -3.2      |  42.45    |
|  24       | -3.2      |  66.35    |
|  25       | -3.2      |  81.83    |


# Random Forest

In [134]:
# SKLearn Random Forest
rf_params = {'n_estimators':(10,150), 'max_depth':(10,200), 'max_features':(2, 30)}

def cv_mse_rf(n_estimators,max_depth, max_features):
    """ 
    Performs cross validation for Random Forest Regressor. To be used for Bayesian optimiser maximizer function.
    
    Parameters
    ----------
    n_estimators : float
        Number of estimators for random forest
    max_depth : float
        Max depth of trees in random forest
    max_features : float
        Max number of features in random forest

    Returns
    -------
    float
        Cross validation score based on negative mean squared error.
        
    """
    # Convert chosen hyperparams to discrete integer
    max_depth = int(max_depth)
    max_features = int(max_features)
    n_estimators = int(n_estimators)
    
    estimator = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, max_features = max_features)

    # Note that neg_mean_squared_error is opposite, thus a negative sign is added for Minimisation Optimisation via BayesOpt
    return cross_validate(estimator, X_train_trans, y_train, cv = 10, scoring = "neg_root_mean_squared_error")["test_score"].mean()

In [135]:
optimizer_rf = BayesianOptimization(cv_mse_rf, rf_params)
optimizer_rf.maximize(n_iter = 60)

|   iter    |  target   | max_depth | max_fe... | n_esti... |
-------------------------------------------------------------
|  1        | -2.714    |  14.78    |  21.4     |  41.23    |
|  2        | -2.717    |  15.92    |  20.51    |  36.09    |
|  3        | -2.688    |  59.09    |  11.31    |  65.41    |
|  4        | -2.711    |  72.22    |  24.64    |  80.11    |
|  5        | -2.693    |  88.84    |  19.55    |  62.83    |
|  6        | -2.729    |  11.14    |  2.375    |  149.3    |
|  7        | -2.704    |  99.85    |  2.073    |  148.1    |
|  8        | -2.837    |  97.81    |  2.188    |  12.58    |
|  9        | -2.748    |  100.0    |  2.0      |  90.51    |
|  10       | -2.683    |  100.0    |  30.0     |  150.0    |
|  11       | -2.738    |  10.0     |  2.0      |  69.23    |
|  12       | -2.716    |  10.0     |  30.0     |  150.0    |
|  13       | -2.747    |  62.54    |  29.89    |  29.18    |
|  14       | -2.693    |  57.42    |  26.52    |  149.8    |
|  15   

# XGBoost

In [136]:
# SKLearn Random Forest
xgb_params = {'n_estimators':(10, 150), 'max_depth':(10, 200), 'learning_rate':(0, 1),
              'subsample':(0, 1), 'gamma':(0, 50), 'reg_alpha':(0, 100), 'reg_lambda':(0, 100)}

def cv_mse_xgb(n_estimators, max_depth, learning_rate, subsample, gamma, reg_alpha, reg_lambda):
    """ 
    Performs cross validation for Random Forest Regressor. To be used for Bayesian optimiser maximizer function.
    
    Parameters
    ----------
    n_estimators : float
        Number of estimators
    max_depth : float
        Max depth of trees
    learning_rate : float
        Learning rate
    subsample : float
        Subsample ratio of training instances 
    gamma : float
        Min loss reduction to make further partition on leaf node   
    reg_alpha : float
        L1 regularisation
    reg_lambda : float
        L2 regularisation

    Returns
    -------
    float
        Cross validation score based on negative mean squared error.
        
    """
    # Convert chosen hyperparams to discrete integer
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    
    estimator = xgb.XGBRegressor(objective='reg:squarederror',
                                 n_estimators = n_estimators, 
                                 max_depth = max_depth, 
                                 learning_rate = learning_rate, 
                                 subsample = subsample,
                                 gamma = gamma, 
                                 reg_alpha = reg_alpha, 
                                 reg_lambda = reg_lambda)

    # Note that neg_mean_squared_error is opposite, thus a negative sign is added for Minimisation Optimisation via BayesOpt
    return cross_validate(estimator, X_train_trans, y_train, cv = 10, scoring = "neg_root_mean_squared_error")["test_score"].mean()

In [141]:
# Warnings due to some current issue with xgboost incompatibility with pandas deprecation
# Fix will be for upcoming xgboost version 1.0.0, but latest version is only 0.90
# See https://github.com/dmlc/xgboost/issues/4300
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Fixed random state due to xgboost hyper param combination throwing a result for NaN
optimizer_xgb = BayesianOptimization(cv_mse_xgb, xgb_params, random_state = 1)
optimizer_xgb.maximize(n_iter = 100)

|   iter    |  target   |   gamma   | learni... | max_depth | n_esti... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        | -2.909    |  20.85    |  0.7203   |  10.01    |  52.33    |  14.68    |  9.234    |  0.1863   |
|  2        | -2.729    |  17.28    |  0.3968   |  58.49    |  68.69    |  68.52    |  20.45    |  0.8781   |
|  3        | -2.961    |  1.369    |  0.6705   |  47.56    |  88.22    |  14.04    |  19.81    |  0.8007   |
|  4        | -4.359    |  48.41    |  0.3134   |  72.31    |  132.7    |  89.46    |  8.504    |  0.03905  |
|  5        | -2.728    |  8.492    |  0.8781   |  18.85    |  68.96    |  95.79    |  53.32    |  0.6919   |
|  6        | -11.79    |  0.0      |  0.0      |  100.0    |  10.0     |  42.16    |  100.0    |  1.0      |
|  7        | -2.744    |  11.43    |  0.8313   |  19.31    |  76.88    |  57.02    |  23.01    |  0.5994   |
|  8      

# LightGBM


In [217]:
round(1.23, 3)

1.23

In [226]:
# SKLearn Random Forest
lgbm_params = {'n_estimators':(10, 150), 'max_depth':(10, 200), 'learning_rate':(0.001, 1),
               'reg_alpha':(0, 100), 'reg_lambda':(0, 100)}

def cv_mse_lgbm(n_estimators, max_depth, learning_rate, reg_alpha, reg_lambda):
    """ 
    Performs cross validation for Random Forest Regressor. To be used for Bayesian optimiser maximizer function.
    
    Parameters
    ----------
    n_estimators : float
        Number of estimators
    max_depth : float
        Max depth of trees
    learning_rate : float
        Learning rate
    reg_alpha : float
        L1 regularisation
    reg_lambda : float
        L2 regularisation

    Returns
    -------
    float
        Cross validation score based on negative mean squared error.
        
    """
    # Convert chosen hyperparams to discrete integer
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    
    estimator = lgbm.LGBMRegressor(n_estimators = n_estimators, 
                                 max_depth = max_depth, 
                                 learning_rate = learning_rate, 
                                 reg_alpha = reg_alpha, 
                                 reg_lambda = reg_lambda)
    
    # Note that neg_mean_squared_error is opposite, thus a negative sign is added for Minimisation Optimisation via BayesOpt
    return cross_validate(estimator, X_train_trans, y_train, cv = 10, scoring = "neg_root_mean_squared_error")["test_score"].mean()

In [227]:
# Fixed random state due to xgboost hyper param combination throwing a result for NaN
optimizer_lgbm = BayesianOptimization(cv_mse_lgbm, lgbm_params, random_state = 1)
optimizer_lgbm.maximize(n_iter = 100)

|   iter    |  target   | learni... | max_depth | n_esti... | reg_alpha | reg_la... |
-------------------------------------------------------------------------------------
|  1        | -2.727    |  0.4176   |  74.83    |  10.02    |  30.23    |  14.68    |
|  2        | -2.749    |  0.09325  |  26.76    |  58.38    |  39.68    |  53.88    |
|  3        | -2.809    |  0.4198   |  71.67    |  38.62    |  87.81    |  2.739    |
|  4        | -2.802    |  0.6708   |  47.56    |  88.22    |  14.04    |  19.81    |
|  5        | -2.79     |  0.8009   |  97.14    |  53.88    |  69.23    |  87.64    |
|  6        | -2.769    |  1.0      |  10.0     |  10.0     |  4.7e-11  |  100.0    |
|  7        | -3.167    |  0.001    |  10.0     |  150.0    |  100.0    |  100.0    |
|  8        | -3.19     |  0.001    |  10.0     |  10.0     |  0.0      |  0.0      |
|  9        | -3.136    |  0.001    |  100.0    |  150.0    |  0.0      |  100.0    |
|  10       | -2.823    |  1.0      |  10.0     |  10.

In [229]:
# Compare the best models
cv_rmse = [-optimizer_lmlasso.max['target'], 
             -optimizer_lmridge.max['target'], 
             -optimizer_rf.max['target'],
             -optimizer_xgb.max['target'],
             -optimizer_lgbm.max['target']]

models = ["lm_lasso", "lm_ridge", "randomforest", "xgb", "lgbm"]

cv_df = pd.DataFrame(cv_rmse, index = models, columns = ["cv_score"])

cv_df

,cv_score
lm_lasso,2.756544
lm_ridge,2.735042
randomforest,2.627075
xgb,2.727759
lgbm,2.726576


In [265]:
# Adjusting discrete hyperparam for certain models
rf_hyperparam = optimizer_rf.max['params']
rf_hyperparam['max_depth'] = int(rf_hyperparam['max_depth'])
rf_hyperparam['max_features'] = int(rf_hyperparam['max_features'])
rf_hyperparam['n_estimators'] = int(rf_hyperparam['n_estimators'])

xgb_hyperparam = optimizer_xgb.max['params']
xgb_hyperparam['max_depth'] = int(xgb_hyperparam['max_depth'])
xgb_hyperparam['n_estimators'] = int(xgb_hyperparam['n_estimators'])

lgbm_hyperparam = optimizer_lgbm.max['params']
lgbm_hyperparam['max_depth'] = int(lgbm_hyperparam['max_depth'])
lgbm_hyperparam['n_estimators'] = int(lgbm_hyperparam['n_estimators'])

In [236]:
# Store as Series for writing to csv
lmlasso_hyperparam = pd.Series(optimizer_lmlasso.max['params'])

lmridge_hyperparam = pd.Series(optimizer_lmridge.max['params'])

rf_hyperparam = pd.Series(rf_hyperparam)

xgb_hyperparam = pd.Series(xgb_hyperparam)

lgbm_hyperparam = pd.Series(lgbm_hyperparam)

In [237]:
# Output to csv
cv_df.to_csv("../data/output/cv_results.csv")

lmlasso_hyperparam.to_csv("../data/output/lmlasso_hyperparam.csv")
lmridge_hyperparam.to_csv("../data/output/lmridge_hyperparam.csv")
rf_hyperparam.to_csv("../data/output/rf_hyperparam.csv")
xgb_hyperparam.to_csv("../data/output/xgb_hyperparam.csv")
lgbm_hyperparam.to_csv("../data/output/lgbm_hyperparam.csv")

# Test Data

In [238]:
# Test set
test_data  = pd.read_csv("../data/processed/test.csv")

X_test = test_data.drop(["G3"], axis = 1)
y_test = test_data["G3"]

In [252]:
# Convert to dataframe
X_test_trans = pd.DataFrame(preprocessor.fit_transform(X_test),
                            index = X_test.index,
                             columns = (list(numeric_features) +
                                       list(preprocessor.named_transformers_['ohe'].get_feature_names(categorical_features))))

In [260]:
best_lasso = Lasso().set_params(**lmlasso_hyperparam)
best_lasso.fit(X_train_trans, y_train)

Lasso(alpha=0.0039689085736815954, copy_X=True, fit_intercept=True,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [261]:
best_ridge = Ridge().set_params(**lmridge_hyperparam)
best_ridge.fit(X_train_trans, y_train)

Ridge(alpha=61.716008089193764, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [262]:
best_rf = RandomForestRegressor().set_params(**rf_hyperparam)
best_rf.fit(X_train_trans, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=63, max_features=7, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=114, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [269]:
best_xgb = XGBRegressor().set_params(**xgb_hyperparam)
best_xgb.fit(X_train_trans, y_train)

[00:24:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=8.491520978228445,
             importance_type='gain', learning_rate=0.8781425034294131,
             max_delta_step=0, max_depth=18, min_child_weight=1, missing=None,
             n_estimators=68, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=95.7889530150502,
             reg_lambda=53.316528497301704, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.6918771139504734, verbosity=1)

In [270]:
best_lgbm = lgbm.LGBMRegressor().set_params(**lgbm_hyperparam)
best_lgbm.fit(X_train_trans, y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.41760498269787144,
              max_depth=74, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=10, n_jobs=-1, num_leaves=31,
              objective=None, random_state=None, reg_alpha=30.233257263183976,
              reg_lambda=14.675589081711305, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

# Test Data Prediction

In [280]:
test_rmse = []
test_rmse.append(np.sqrt(mean_squared_error(y_test, best_lasso.predict(X_test_trans))))
test_rmse.append(np.sqrt(mean_squared_error(y_test, best_ridge.predict(X_test_trans))))
test_rmse.append(np.sqrt(mean_squared_error(y_test, best_rf.predict(X_test_trans))))
test_rmse.append(np.sqrt(mean_squared_error(y_test, best_xgb.predict(X_test_trans))))
test_rmse.append(np.sqrt(mean_squared_error(y_test, best_lgbm.predict(X_test_trans))))

# Convert to Dataframe
test_rmse = pd.DataFrame(test_rmse, index= models, columns = ["test_rmse"])

In [281]:
test_rmse

,test_rmse
lm_lasso,2.483822
lm_ridge,2.481183
randomforest,2.458307
xgb,2.510746
lgbm,2.577540


In [282]:
np.mean(y_test)

12.062015503875969

In [296]:
# Best model
print(test_rmse.idxmin())

test_rmse    randomforest
dtype: object


In [306]:
feat_importance = pd.DataFrame(best_rf.feature_importances_, index = X_train_trans.columns, columns = ["Importance"])
feat_importance = feat_importance.sort_values(by = "Importance", ascending = False).reset_index()

In [307]:
feat_importance

,index,Importance
0,failures,0.097314
1,absences,0.055374
2,school_MS,0.051027
3,age,0.049253
4,goout,0.047025
5,Walc,0.046958
6,Medu,0.046369
7,higher_yes,0.045723
8,freetime,0.038600
9,studytime,0.038511


In [314]:
# https://github.com/nipunbatra/50-ggplot-python/blob/master/Altair/DivergingLollipop.ipynb
import altair as alt
from altair import *
c1 = alt.Chart(feat_importance).mark_bar(color='black').encode(
    y=alt.Y('index', sort=SortField(op='mean', order='descending', field='Importance')),
    x=alt.X('Importance' )
)

SchemaValidationError: Invalid specification

        altair.vegalite.v3.schema.core.SortField, validating 'additionalProperties'

        Additional properties are not allowed ('op' was unexpected)
        

In [ ]:
c2 = Chart(mtcars).mark_circle(color='black', size=400).encode(y=Y('model', sort=SortField(op='mean', order='descending', field='mpg_z')),
                                x=X('mpg_z' ), text='mpg_z').transform_data(
   calculate=[Formula('Performance', expr.where(expr.df.mpg_z < mean_mpg_z,'Below average','Above average'))],
   
).configure_cell(height=100, width=400).configure_scale(bandSize=22)